# Importing Packages

In [8]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import json
from sqlalchemy import create_engine
import itertools

# Load Environment Variables

In [2]:
# Load authentication data in .env file
load_dotenv()

True

# Get Categories Data

In [3]:
def get_categories():
    response = requests.get('https://developers.zomato.com/api/v2.1/categories', headers=HEADERS)
    if response.status_code not in (200, 400):
        raise Exception('Failed to get the data')
    else:
        return response.status_code, response.json()
    
def read_categories(json_data):
    data = {idx:row['categories'] for idx, row in enumerate(json_data['categories'])}
    return pd.DataFrame.from_dict(data, orient='index')

# Get Cities Data

In [4]:
def get_cities(list_cities):
    params = {'q' : ','.join(list_cities)}
    response = requests.get('https://developers.zomato.com/api/v2.1/cities', headers=HEADERS, params=params)
    if response.status_code != 200:
        raise Exception('Failed to get the data')
    else:
        return response.status_code, response.json()

def read_cities(json_data):
    data = {idx:row for idx, row in enumerate(json_data['location_suggestions'])}
    return pd.DataFrame.from_dict(data, orient='index')

# Get Collections Data

In [148]:
def get_collections(list_city_id):
    all_data = []
    for record in list_city_id:
        params = {'city_id' : record, 'count' : 1000}
        response = requests.get('https://developers.zomato.com/api/v2.1/collections', headers=HEADERS, params=params)
        if response.status_code != 200:
            print(response.status_code)
            print(response.text)
            raise Exception('Failed to get the data')
        else:
            all_data.append(response.json())
    return response.status_code, all_data


def read_collections(json_data, list_city_id): 
    data = dict()
    row = 0
    for i in range(len(list_city_id)):
        record = dict()
        for j in range(len(json_data[i]['collections'])):
            record.update({'city_id' : list_city_id[i]})
            record.update(json_data[i]['collections'][j]['collection'])
            data.update({row : record.copy()})
            row += 1
    return pd.DataFrame.from_dict(data, orient='index')

# Get Restaurant Data

In [145]:
def get_search(list_city_id, list_collection_id):
    pass
#     params = {}
#     response = requests.get('https://developers.zomato.com/api/v2.1/search', header=HEADERS, params=params)

def read_search():
    pass

# Connect and Write to PostgreSQL

In [146]:
def connect_db():
    engine = create_engine(f"postgresql+psycopg2://{os.getenv('db-user')}:{os.getenv('db-pwd')}@{os.getenv('db-host')}")
    conn = engine.connect()
    return engine, conn

def close_db(engine, conn):
    conn.close()
    engine.dispose()
    
def save_to_db(data, table_name):
    engine, conn = connect_db()
    data.to_sql(table_name, conn, if_exists='replace', index=False)
    close_db(engine, conn)

# Data Request Main Function

In [152]:
HEADERS = headers = {'user-key' : os.getenv('user-key')}
list_cities = ['Jakarta', 'Bandung', 'Bali']

df_categories = read_categories(get_categories()[1])
df_cities = read_cities(get_cities(list_cities)[1])
df_collections = read_collections(get_collections(df_cities['id'])[1], df_cities['id'])

save_to_db(df_collections, 'collections')